In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 3.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import sklearn
import keras_tuner as kt
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# split training into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
def BaseClassification():
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(32,32,3)))
  model.add(keras.layers.Dense(units=10, activation="softmax"))

  opt = keras.optimizers.Adam(learning_rate=0.01)
  model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

  return model

In [ ]:
model = BaseClassification()

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20)

Epoch 1/20
1250/1250 [==============================] - 6s 4ms/step - loss: 1.9631 - accuracy: 0.2980 - val_loss: 1.8718 - val_accuracy: 0.3392
Epoch 2/20
1250/1250 [==============================] - 3s 3ms/step - loss: 1.8577 - accuracy: 0.3438 - val_loss: 1.9037 - val_accuracy: 0.3357
Epoch 3/20
1250/1250 [==============================] - 3s 3ms/step - loss: 1.8230 - accuracy: 0.3615 - val_loss: 1.8111 - val_accuracy: 0.3642
Epoch 4/20
1250/1250 [==============================] - 4s 3ms/step - loss: 1.8029 - accuracy: 0.3749 - val_loss: 1.7976 - val_accuracy: 0.3687
Epoch 5/20
1250/1250 [==============================] - 3s 3ms/step - loss: 1.7893 - accuracy: 0.3767 - val_loss: 1.7996 - val_accuracy: 0.3709
Epoch 6/20
1250/1250 [==============================] - 4s 3ms/step - loss: 1.7810 - accuracy: 0.3794 - val_loss: 1.7825 - val_accuracy: 0.3824
Epoch 7/20
1250/1250 [==============================] - 4s 3ms/step - loss: 1.7712 - accuracy: 0.3836 - val_loss: 1.8278 - val_accuracy:

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.7593 - accuracy: 0.3857


[1.7593483924865723, 0.385699987411499]

In [ ]:
class MyHyperModel(kt.HyperModel):
  def build(self, hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(32,32,3)))

    hp_l2reg = hp.Float("l2reg", min_value=0.001, max_value=10, step=10, sampling="log")
    model.add(keras.layers.Dense(units=10, activation="softmax", activity_regularizer=keras.regularizers.L2(hp_l2reg)))

    hp_learning_rate = hp.Float("learning_rate", min_value=0.0001, max_value=0.1, step=10, sampling="log")

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model
  
  def fit(self, hp, model, *args, **kwargs):
    return model.fit(
        *args,
        batch_size=hp.Int("batch_size", 32, 256, step=32),
        **kwargs,
    )


In [ ]:
tuner = kt.Hyperband(MyHyperModel(), objective='val_accuracy', overwrite=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[stop_early])

Trial 223 Complete [00h 00m 37s]
val_accuracy: 0.2615000009536743

Best val_accuracy So Far: 0.3995000123977661
Total elapsed time: 00h 32m 38s


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0145 summary
Hyperparameters:
l2reg: 0.01
learning_rate: 0.001
batch_size: 192
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: 0138
Score: 0.39662501215934753

Trial 0132 summary
Hyperparameters:
l2reg: 0.001
learning_rate: 0.001
batch_size: 224
tuner/epochs: 12
tuner/initial_epoch: 4
tuner/bracket: 4
tuner/round: 2
tuner/trial_id: 0103
Score: 0.39262500405311584

Trial 0146 summary
Hyperparameters:
l2reg: 0.01
learning_rate: 0.001
batch_size: 192
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 0145
Score: 0.3917500078678131

Trial 0142 summary
Hyperparameters:
l2reg: 0.001
learning_rate: 0.001
batch_size: 224
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: 0132
Score: 0.39149999618530273

Trial 0144 summary
Hyperparameters:
l2reg: 0.001
lear

In [ ]:
best_model = tuner.get_best_models()[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 10)                30730     
                                                                 
Total params: 30,730
Trainable params: 30,730
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.7514 - accuracy: 0.3908


[1.7514220476150513, 0.39079999923706055]